<a href="https://colab.research.google.com/github/vitaldb/examples/blob/master/table1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stat

In [28]:
# read data
df = pd.read_csv('https://api.vitaldb.net/cases')

# add columns
df['opdur'] = df['opend'] - df['opstart']
df['anedur'] = df['aneend'] - df['anestart']
df['hospdur'] = df['dis'] - df['adm']

# remove columns
df.drop(columns=['opstart', 'opend', 'anestart', 'aneend', 'dis', 'adm'], inplace=True)
df = df.loc[:, ~df.columns.str.endswith('id')]

# group variable
grpvar = 'death_inhosp'

# manually set categorical variables
catvars = ['department']

In [43]:
def format_pval(pval):
    '''Returns the formated string of a p-value'''
    if pval < 0.001:
        return '< 0.001'
    return f'{pval:.3f}'

def format_number(num, prec=3):
    '''Returns the formated number up to specific precision'''
    fmt = '{:,.' + str(prec) + 'f}'
    s = fmt.format(num)
    return s.rstrip('0').rstrip('.')

!pip install fexact
import fexact  # https://github.com/boussoffara/fexact
def fisher_exact(table):
    '''Returns p-value for the Fisher's exact test of nxm contingency table
    fisher_exact([[8,2,12], [1,5,2]])  # 0.011825369366598752
    '''
    return fexact.fexact(np.array(table))

In [47]:
import warnings

# convert boolean type to int
df.replace({False: 0, True: 1}, inplace=True)

if grpvar is not None:
    grp_names = np.unique(df[grpvar])

# for csv file
lines = []

# Generate table header
tabs = ['', 'Total']
if grpvar is not None:
    for grp_name in grp_names:
        tabs.append(f'{grpvar}={grp_name}')
    if len(grp_names) > 1:
        tabs.append('P-value')
        tabs.append('Test')
lines.append(tabs)

tabs = ['n']
if grpvar is not None:
    tabs.append(format_number(sum(~df[grpvar].isnull())))
    for grp_name in grp_names:
        tabs.append(format_number(sum(df[grpvar] == grp_name)) + ' (' + format_number(np.mean(df[grpvar] == grp_name) * 100, 1) + '%)')
else:
    tabs.append(format_number(len(df)))
lines.append(tabs)

# Generate statistics for each variable
for prop in df.columns:
    if prop == grpvar:
        continue
    try:
        pd.to_numeric(df[prop])
        isstr = False
    except:
        isstr = True

    unique_values = sorted(df.loc[~df[prop].isnull(), prop].unique())  # unique values

    iscat = len(unique_values) < 8
    if prop in catvars:
        iscat = True

    if isstr and not iscat:
        continue

    if iscat:  # categorical variables --> represents as count (percent)
        if grpvar is not None:  # create cross table (value x grp)
            xtab = pd.crosstab(df[prop], df[grpvar]).fillna(0)
            pval = None
            if len(grp_names) > 1: # NEJM requires Exact method for all categorical variables
                if (xtab > 5).all(axis=None):  # if there is an incidence < 5
                    pval = stat.chi2_contingency(xtab)[1]
                    test_name = 'Chi-square'
                else:
                    pval = fisher_exact(xtab.T.values)
                    test_name = 'Fisher\'s exact'

        is_binary = (len(unique_values) == 2) and (unique_values[0] == 0 and unique_values[1] == 1)
        if is_binary:  # binary
            # print total
            tabs = [prop, format_number(sum(df[prop] == 1)) + ' (' + format_number(np.mean(df[prop] == 1) * 100, 1) + '%)']
            if grpvar is not None: # print group values
                for grp_name in grp_names:
                    grp_mask = (df[grpvar] == grp_name)
                    tabs.append(format_number(sum(df.loc[grp_mask, prop])) + ' (' + format_number(np.mean(df.loc[grp_mask, prop])*100, 1) + '%)')
                if pval is not None:
                    tabs.append(format_pval(pval))
                    tabs.append(test_name)
            lines.append(tabs)
        else:
            for uval in unique_values:
                # print total
                tabs = [f'{prop}={uval}', format_number(sum(df[prop] == uval)) + ' (' + format_number(np.mean(df[prop] == uval) * 100, 1) + '%)']
                if grpvar is not None: # print group values
                    for grp_name in grp_names:
                        grp_mask = (df[grpvar] == grp_name)
                        tabs.append(format_number(sum(df.loc[grp_mask, prop] == uval)) + ' (' + format_number(np.mean(df.loc[grp_mask, prop] == uval) * 100, 1) + '%)')
                    if pval is not None:
                        if uval == unique_values[0]:
                            tabs.append(format_pval(pval))
                            tabs.append(test_name)
                lines.append(tabs)

    else:  # continuous variables --> represents as mean (SD)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            isnorm = stat.shapiro(df[prop])[1] > 0.05  # check if it is normal distribution
        if isnorm:  # normal distribution
            # print total
            m = df[prop].mean()
            s = df[prop].std()
            tabs = [prop, f'{m:.3f} ({s:.3f})']

            if grpvar is not None:
                # extract group values
                grp_vals = []
                for grp_name in grp_names:
                    a = df.loc[df[grpvar] == grp_name, prop]
                    grp_vals.append(a[~a.isnull()])

                # print group values
                for igrp in range(len(grp_vals)):
                    m = grp_vals[igrp].mean()
                    s = grp_vals[igrp].std()
                    tabs.append(f'{m:.3f} ({s:.3f})')

                # print stat
                if len(grp_names) == 2:
                    equal_var = stat.levene(grp_vals[0], grp_vals[1])[1] > 0.05  # levene
                    pval = stat.ttest_ind(grp_vals[0], grp_vals[1], equal_var=equal_var)[1]
                    test_name = 'T-test'
                else:  # 3 or more groups -> anova
                    equal_var = stat.levene(*grp_vals)[1] > 0.05  # levene + homoscedasticity
                    if equal_var:
                        pval = stat.f_oneway(*grp_vals)[1]
                        test_name = 'One-way ANOVA'
                    else:
                        pval = stat.kruskal(*grp_vals)[1]
                        test_name = 'Kruskal-Wallis'
                    tabs.append(format_pval(pval))
                    tabs.append(test_name)
        else:  # non-normal
            # print total
            m = df[prop].median()
            q1 = df[prop].quantile(0.25)
            q2 = df[prop].quantile(0.75)
            tabs = [prop, format_number(m, 3) + ' (' + format_number(q1, 3) + '-' + format_number(q2, 3) + ')']

            if grpvar is not None:
                # extract group values
                grp_vals = []
                for grp_name in grp_names:
                    a = df.loc[df[grpvar] == grp_name, prop]
                    grp_vals.append(a[~a.isnull()])

                # print group value
                for igrp in range(len(grp_vals)):
                    m = grp_vals[igrp].mean()
                    q1 = grp_vals[igrp].quantile(0.25)
                    q2 = grp_vals[igrp].quantile(0.75)
                    tabs.append(format_number(m, 3) + ' (' + format_number(q1, 3) + '-' + format_number(q2, 3) + ')')

                # print stat
                if len(grp_vals) == 2:
                    pval = stat.mannwhitneyu(grp_vals[0], grp_vals[1], alternative='two-sided')[1]
                    test_name = 'Mann-Whitney'
                elif len(grp_vals) > 2:  # > 3 groups
                    pval = stat.kruskal(*grp_vals)[1]
                    test_name = 'Kruskal-Wallis'
                tabs.append(format_pval(pval))
                tabs.append(test_name)

        lines.append(tabs)

for tabs in lines:
    print('\t'.join(tabs))

pd.DataFrame(lines).to_csv('table1.csv', index=False, header=False)

	Total	death_inhosp=0	death_inhosp=1	P-value	Test
n	6,388	6,331 (99.1%)	57 (0.9%)
casestart=0	6,388 (100%)	6,331 (100%)	57 (100%)	1.000	Chi-square
caseend	9,924.5 (6,194.5-15,072.75)	11,342.646 (6,189-15,053)	12,019 (6,655-16,200)	0.380	Mann-Whitney
icu_days	0 (0-0)	0.456 (0-0)	11.281 (0-16)	< 0.001	Mann-Whitney
age	59 (48-68)	57.309 (48-68)	55.893 (47-73)	0.452	Mann-Whitney
sex=F	3,145 (49.2%)	3,125 (49.4%)	20 (35.1%)	0.044	Chi-square
sex=M	3,243 (50.8%)	3,206 (50.6%)	37 (64.9%)
height	162.2 (156.1-168.7)	162.25 (156.1-168.7)	155.37 (158.2-171)	0.392	Mann-Whitney
weight	60.5 (53.3-68.7)	61.508 (53.35-68.675)	58.916 (51.3-71.2)	0.791	Mann-Whitney
bmi	23.1 (20.9-25.4)	23.28 (20.9-25.4)	23.147 (20.7-25.3)	0.737	Mann-Whitney
asa=1.0	1,792 (28.1%)	1,783 (28.2%)	9 (15.8%)	< 0.001	Fisher's exact
asa=2.0	3,699 (57.9%)	3,682 (58.2%)	17 (29.8%)
asa=3.0	703 (11%)	686 (10.8%)	17 (29.8%)
asa=4.0	48 (0.8%)	36 (0.6%)	12 (21.1%)
asa=6.0	13 (0.2%)	13 (0.2%)	0 (0%)
emop	782 (12.2%)	753 (11.9%)	29 (50.9